In [ ]:
import ollama
from langchain_ollama import OllamaLLM, OllamaEmbeddings
from langchain_community.vectorstores import PGVector
from langchain_core.documents import Document
import pandas as pd

# --- 1. กำหนดค่า Ollama ---
# ใช้ OllamaLLM เมื่อคุณต้องการแค่ดึงข้อความ ไม่ได้ใช้คุณสมบัติ Chat Model เต็มรูปแบบ
model = OllamaLLM(model="llama3.2", base_url="http://localhost:11434")
embeddings = OllamaEmbeddings(model="mxbai-embed-large", base_url="http://localhost:11434")

# --- 2. กำหนดค่าการเชื่อมต่อ PostgreSQL ---
DB_USER = "myuser"
DB_PASSWORD = "mypassword"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "mydatabase"
CONNECTION_STRING = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
COLLECTION_NAME = "my_ollama_documents_collection"

# # --- 3. ดึงข้อมูลจากไฟล์ CSV และเตรียม LangChain Documents ---
# csv_file_path = "my_doc.csv" # ตรวจสอบพาธของไฟล์ CSV ของคุณ

try:
    df = pd.read_csv(csv_file_path, encoding='utf-8')
except FileNotFoundError:
    print(f"Error: File not found at {csv_file_path}")
    exit()

docs = []
# กำหนดชื่อคอลัมน์ที่เป็นเนื้อหาหลักที่คุณต้องการให้ AI ใช้ค้นหา
# *** ต้องตรงกับชื่อคอลัมน์ในไฟล์ CSV ของคุณเป๊ะๆ ***
content_column = 'คำถาม' 

for index, row in df.iterrows():
    # ตรวจสอบว่าคอลัมน์เนื้อหาหลักไม่ว่าง
    if pd.isna(row[content_column]):
        continue # ข้ามแถวที่ไม่มีเนื้อหาหลัก

    page_content = str(row[content_column])
    
    # สร้าง metadata จากคอลัมน์อื่นๆ ทั้งหมด ยกเว้นคอลัมน์เนื้อหาหลัก
    # ในกรณีนี้ 'คำตอบ' จะถูกเก็บใน metadata ของเอกสาร
    metadata = row.drop(content_column).to_dict() 

    docs.append(Document(page_content=page_content, metadata=metadata))

print(f"Loaded {len(docs)} documents from CSV.")

# # --- 4. สร้าง Vector Store และบันทึกข้อมูลลง PostgreSQL ---
# print("Connecting to PGVector store and adding documents...")
# # 'pre_delete_collection=True' จะลบ collection เก่าก่อนเพิ่มใหม่เสมอ
vectorstore = PGVector.from_documents(
    documents=docs,
    embedding=embeddings,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
    pre_delete_collection=True # ตั้งค่าเป็น True ถ้าต้องการลบข้อมูลเก่าใน collection ก่อนเพิ่มใหม่
)
print(f"Documents added to PostgreSQL in collection '{COLLECTION_NAME}'.")

# --- 5. ใช้ Vector Store เป็น Retriever และค้นหา ---
# คุณสามารถปรับ k เพื่อดึงจำนวนเอกสารที่ต้องการมาตรวจสอบได้
retriever = vectorstore.as_retriever(search_kwargs={"k": 1}) 

# ทำการค้นหาข้อความที่คล้ายคลึงที่สุด
query_text = "ขอเปลี่ยนรหัสผ่าน" # คำถามที่คุณต้องการใช้ค้นหา
retrieved_documents = retriever.invoke(query_text)

print(f"\n--- Retrieved Documents for query: '{query_text}' ---")
if retrieved_documents:
    # แสดงเนื้อหาของเอกสารที่ถูกดึงมา (ซึ่งตอนนี้คือ 'ประโยคขอแก้ไขข้อมูลอีเมล')
    print(f"Content (จากคอลัมน์ 'คำถาม'): {retrieved_documents[0].page_content}")
    
    # แสดง metadata ซึ่งจะรวมถึงคอลัมน์ 'คำตอบ' ที่เราต้องการดู
    print(f"Metadata (รวมถึง 'คำตอบ'): {retrieved_documents[0].metadata}")
    
    # แสดง 'คำตอบ' ที่ตรงกันจาก metadata โดยตรง
    if 'คำตอบ' in retrieved_documents[0].metadata:
        print(f"**คำตอบที่เกี่ยวข้อง:** {retrieved_documents[0].metadata['คำตอบ']}")
    else:
        print("ไม่พบคอลัมน์ 'คำตอบ' ใน metadata ของเอกสารที่ดึงมา")
else:
    print("ไม่พบเอกสารสำหรับคำถามนี้")

print("\nProcess completed.")

: 

In [ ]:
import ollama
from langchain_ollama import OllamaLLM, OllamaEmbeddings, ChatOllama # เพิ่ม ChatOllama
from langchain_community.vectorstores import PGVector
from langchain_core.documents import Document
import pandas as pd
from langchain_core.prompts import ChatPromptTemplate # เพิ่ม
from langchain_core.output_parsers import StrOutputParser # เพิ่ม
from langchain_core.runnables import RunnablePassthrough # เพิ่ม

# --- 1. กำหนดค่า Ollama ---
# เปลี่ยนเป็น ChatOllama เพื่อให้รองรับ ChatPromptTemplate และการสร้างคำตอบแบบสนทนา
model = ChatOllama(model="llama3.2", base_url="http://localhost:11434")
embeddings = OllamaEmbeddings(model="mxbai-embed-large", base_url="http://localhost:11434")

# --- 2. กำหนดค่าการเชื่อมต่อ PostgreSQL ---
DB_USER = "myuser"
DB_PASSWORD = "mypassword"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "mydatabase"
CONNECTION_STRING = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
COLLECTION_NAME = "my_ollama_documents_collection"

# --- 3. ดึงข้อมูลจากไฟล์ CSV และเตรียม LangChain Documents ---
csv_file_path = "my_doc.csv" # ตรวจสอบพาธของไฟล์ CSV ของคุณ

try:
    df = pd.read_csv(csv_file_path, encoding='utf-8')
except FileNotFoundError:
    print(f"Error: File not found at {csv_file_path}")
    exit()

docs = []
# กำหนดชื่อคอลัมน์ที่เป็นเนื้อหาหลักที่คุณต้องการให้ AI ใช้ค้นหา
# *** ต้องตรงกับชื่อคอลัมน์ในไฟล์ CSV ของคุณเป๊ะๆ ***
content_column = 'คำถาม' 

for index, row in df.iterrows():
    # ตรวจสอบว่าคอลัมน์เนื้อหาหลักไม่ว่าง
    if pd.isna(row[content_column]):
        continue # ข้ามแถวที่ไม่มีเนื้อหาหลัก

    page_content = str(row[content_column])
    
    # สร้าง metadata จากคอลัมน์อื่นๆ ทั้งหมด ยกเว้นคอลัมน์เนื้อหาหลัก
    # ในกรณีนี้ 'คำตอบ' จะถูกเก็บใน metadata ของเอกสาร
    metadata = row.drop(content_column).to_dict() 

    docs.append(Document(page_content=page_content, metadata=metadata))

print(f"Loaded {len(docs)} documents from CSV.")

# --- 4. สร้าง Vector Store และบันทึกข้อมูลลง PostgreSQL ---
print("Connecting to PGVector store and adding documents...")
# 'pre_delete_collection=True' จะลบ collection เก่าก่อนเพิ่มใหม่เสมอ
vectorstore = PGVector.from_documents(
    documents=docs,
    embedding=embeddings,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
    pre_delete_collection=True # ตั้งค่าเป็น True ถ้าต้องการลบข้อมูลเก่าใน collection ก่อนเพิ่มใหม่
)
print(f"Documents added to PostgreSQL in collection '{COLLECTION_NAME}'.")

# --- 5. ใช้ Vector Store เป็น Retriever และสร้าง RAG Chain ---
# คุณสามารถปรับ k เพื่อดึงจำนวนเอกสารที่ต้องการมาใช้เป็นบริบทได้ (เช่น k=3, k=5)
# ถ้าอยากให้ LLM ได้ context จาก 'คำถาม' และ 'คำตอบ' ให้ retriever ดึง metadata มาครบ
retriever = vectorstore.as_retriever(search_kwargs={"k": 1}) 

# สร้าง Prompt Template สำหรับ RAG Chain
# เราจะส่งทั้ง 'คำถาม' (page_content) และ 'คำตอบ' (จาก metadata) ไปให้ LLM ใช้เป็นบริบท
template = """คุณคือผู้ช่วยที่ตอบคำถามอย่างเป็นมิตรและเป็นประโยชน์

อ้างอิงจากข้อมูลบริบท (context) ที่ให้มาเท่านั้น หากไม่พบข้อมูลในบริบท ให้ระบุว่าไม่สามารถตอบคำถามจากข้อมูลที่มีได้

Context (ข้อมูลที่เกี่ยวข้องจากฐานความรู้):
{context}

Question (คำถามจากผู้ใช้):
{question}

Answer (คำตอบ):
"""
prompt = ChatPromptTemplate.from_template(template)

# ---
# ปรับส่วน context ใน rag_chain เพื่อส่งทั้ง 'คำถาม' (page_content) และ 'คำตอบ' (metadata['คำตอบ'])
# ไปให้ LLM เพื่อให้มีบริบทที่ครบถ้วนยิ่งขึ้นในการสร้างคำตอบ
# ---
def format_docs_for_context(docs):
    formatted_string = ""
    for doc in docs:
        # ดึง 'คำตอบ' จาก metadata
        answer_content = doc.metadata.get('คำตอบ', 'ไม่พบข้อมูลคำตอบ')
        # page_content คือ 'คำถาม' ที่เราใช้ในการ embed
        question_content = doc.page_content 
        
        formatted_string += f"คำถาม: {question_content}\n"
        formatted_string += f"คำตอบ: {answer_content}\n\n"
    return formatted_string.strip() # ลบช่องว่างส่วนเกินที่ท้าย

rag_chain = (
    {"context": retriever | format_docs_for_context, # ดึงเอกสารแล้วนำไปจัดรูปแบบเป็น context
     "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# --- 6. ใช้ RAG Chain เพื่อรับคำตอบจาก AI ---
query_text = "วิธีเปลี่ยนรหัสผ่าน" # คำถามที่คุณต้องการใช้ค้นหา
print(f"\n--- Generating Answer for query: '{query_text}' ---")
final_answer = rag_chain.invoke(query_text)

print(final_answer)

print("\nProcess completed.")

Loaded 200 documents from CSV.
Connecting to PGVector store and adding documents...


c:\Users\stamp7ven\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_community\vectorstores\pgvector.py:490: LangChainPendingDeprecationWarning: Please use JSONB instead of JSON for metadata. This change will allow for more efficient querying that involves filtering based on metadata. Please note that filtering operators have been changed when using JSONB metadata to be prefixed with a $ sign to avoid name collisions with columns. If you're using an existing database, you will need to create a db migration for your metadata column to be JSONB and update your queries to use the new operators. 
  store = cls(


Documents added to PostgreSQL in collection 'my_ollama_documents_collection'.

--- Generating Answer for query: 'วิธีเปลี่ยนรหัสผ่าน' ---
ขอโทษครับ/ค่ะ ผมไม่สามารถช่วยคุณเปลี่ยนรหัสผ่านได้ครับ/ค่ะ เนื่องจากการเปลี่ยนรหัสผ่านเป็นกระบวนการที่ต้องทำโดยผู้ใช้ตัวเอง และผมไม่มีข้อมูลที่ถูกต้องเกี่ยวกับการเปลี่ยนรหัสผ่านทางออนไลน์หรือไม่ครับ/ค่ะ

หากคุณต้องการเปลี่ยนรหัสผ่าน คุณสามารถอ้างอิงจากคำตอบที่ให้ไว้ในข้อมูลบริบท (context) ที่มีครับ/ค่ะ โดยทำตามขั้นตอนดังนี้:
1. เข้าสู่ระบบ ด้วยบัญชีผู้ใช้งานปัจจุบัน
2. ไปที่เมนู บัญชี > เปลี่ยนรหัสผ่าน
3. กรอกข้อมูลดังนี้
4. รหัสผ่านเดิม
5. รหัสผ่านใหม่
6. ยืนยันรหัสผ่านใหม่
7. กดปุ่ม บันทึก เพื่อยืนยันการเปลี่ยนรหัสผ่าน
8. ระบบจะแจ้งว่าเปลี่ยนรหัสผ่านสำเร็จครับ/ค่ะ

Process completed.
